# Install & Import

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Ini Projek Machine Learning

In [ ]:
!apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (4,553 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 160772 files and directories currently installed.)
Preparing to unpack .../tesserac

In [ ]:
!apt install subversion

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1
Suggested packages:
  db5.3-util libapache2-mod-svn subversion-tools
The following NEW packages will be installed:
  libapr1 libaprutil1 libserf-1-1 libsvn1 subversion
0 upgraded, 5 newly installed, 0 to remove and 39 not upgraded.
Need to get 2,237 kB of archives.
After this operation, 9,910 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libapr1 amd64 1.6.3-2 [90.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 libaprutil1 amd64 1.6.1-2 [84.4 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libserf-1-1 amd64 1.3.9-6 [44.4 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libsvn1 amd64 1.9.7-4ubuntu1 [1,183 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic/universe amd64 subversion amd64 1.9.7-4ubuntu1 

In [ ]:
!pip install tfkerassurgeon

  Created wheel for tfkerassurgeon: filename=tfkerassurgeon-0.2.1-cp37-none-any.whl size=22040 sha256=b53c36309f87f2a90ebe87703b5b08342e7addf1f4e5413fcb907dee7571f86c
  Stored in directory: /root/.cache/pip/wheels/33/41/28/daca963a6834fecf8cd9a3b207288ba325fdf012a7ebd95715
Successfully built tfkerassurgeon


In [ ]:
import tensorflow as tf
from tensorflow import lite
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import re
import string
import csv
import warnings
warnings.filterwarnings('ignore')

import pytesseract
import shutil
import os
import random
try:
  from PIL import Image
except ImportError:
  import Image

print(tf.__version__)

2.5.0


# Get Data

In [ ]:
# Upload your kaggle.json to colab main directory
! chmod 600 kaggle.json && (ls ~/.kaggle 2>/dev/null || mkdir ~/.kaggle) && mv kaggle.json ~/.kaggle/ && echo 'Done'

Done


In [ ]:
!kaggle datasets download arashnic/book-recommendation-dataset
!unzip -o book-recommendation-dataset.zip
!ls

 55% 13.0M/23.8M [00:00<00:00, 131MB/s]
100% 23.8M/23.8M [00:00<00:00, 152MB/s]
Archive:  book-recommendation-dataset.zip
  inflating: Books.csv               
  inflating: Ratings.csv             
  inflating: Users.csv               
book-recommendation-dataset.zip  Ratings.csv	      sample_data
Books.csv			 regression_model.h5  Users.csv


In [ ]:
!svn checkout https://github.com/Turing-Team-B21-CAP0257/ML/trunk/ImageTest

A    ImageTest/CM-1.jpg
A    ImageTest/CM-2.jpg
A    ImageTest/CM-3.jpg
A    ImageTest/CM-4.jpg
A    ImageTest/DN-1.jpg
A    ImageTest/DN-2.jpg
A    ImageTest/DN-3.jpg
A    ImageTest/GM-1.jpg
A    ImageTest/GM-2.jpg
A    ImageTest/H-1.jpeg
A    ImageTest/H-2.jpg
A    ImageTest/LV-1.jpg
A    ImageTest/LV-2.jpg
A    ImageTest/TDO.jpg
A    ImageTest/WTD-1.jpg
A    ImageTest/WTD-2.jpg
A    ImageTest/WTD-3.jpg
Checked out revision 5.


# Preprocessing

In [ ]:
# fix column name
def columnfix(columnlist):
  columnlist = list(columnlist)
  for i in range(len(columnlist)):
    columnlist[i] = columnlist[i].lower()
    columnlist[i] = columnlist[i].translate(str.maketrans("","",string.punctuation))

  return columnlist

### Rating Section

In [ ]:
df_rate = pd.read_csv('Ratings.csv')
df_rate.columns = columnfix(df_rate.columns)   # fix column name
df_rate

,userid,isbn,bookrating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [ ]:
newrate = df_rate.groupby('isbn').agg(         # Create new dataframe group by isbn and have column from mean rating
  rating=("bookrating", "mean")
)

# newrate.reset_index(inplace=True)
newrate

,rating
isbn,
0330299891,3.0
0375404120,1.5
0586045007,0.0
9022906116,3.5
9032803328,0.0
...,...
cn113107,0.0
ooo7156103,7.0
§423350229,0.0


### Book Section

In [ ]:
df_book = pd.read_csv('Books.csv')
df_book.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [ ]:
df_book.columns = columnfix(df_book.columns)                  # fix column name

df_book = df_book.join(newrate, on="isbn")                    # join df_book with newrate

df_book['rating'] = df_book['rating'].fillna(0)               # fill NaN rating with 0

df_book = df_book[df_book['booktitle'].notna()]               # remove data with NaN book title

df_book.insert(loc=0, column='bookid', value=df_book.index)   # add bookid in first column

df_book = df_book.replace(",", "", regex=True)                # remove comma

df_book

,bookid,isbn,booktitle,bookauthor,yearofpublication,publisher,imageurls,imageurlm,imageurll,rating
0,0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,0.000000
1,1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,4.928571
2,2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,5.000000
3,3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,4.272727
4,4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,0.000000
...,...,...,...,...,...,...,...,...,...,...
271355,271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,7.000000
271356,271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,4.000000
271357,271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,0.000000
271358,271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,0.000000


In [ ]:
df_book.to_csv('newbook.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

### User Section

In [ ]:
df_user = pd.read_csv('Users.csv')
df_user.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [ ]:
df_user = df_user.fillna(0)                   # fill NaN age with zero

df_user.columns = columnfix(df_user.columns)  # fix column name

df_user

,userid,location,age
0,1,"nyc, new york, usa",0.0
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",0.0
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",0.0
...,...,...,...
278853,278854,"portland, oregon, usa",0.0
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",0.0
278856,278857,"knoxville, tennessee, usa",0.0


In [ ]:
df_user.to_csv('newuser.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

### Rating Section (2)

In [ ]:
df_isbn_id = df_book[["isbn", "bookid"]].copy()
df_isbn_id = df_isbn_id.set_index('isbn')
df_isbn_id

,bookid
isbn,
0195153448,0
0002005018,1
0060973129,2
0374157065,3
0393045218,4
...,...
0440400988,271355
0525447644,271356
006008667X,271357


In [ ]:
df_rate = df_rate.join(df_isbn_id, on="isbn")
df_rate = df_rate[df_rate['bookid'].notna()]
df_rate['bookid'] = df_rate['bookid'].astype(int)
df_rate

,userid,isbn,bookrating,bookid
0,276725,034545104X,0,2966
1,276726,0155061224,5,225816
2,276727,0446520802,0,11053
3,276729,052165615X,3,246838
4,276729,0521795028,6,246839
...,...,...,...,...
1149774,276704,0876044011,0,69543
1149775,276704,1563526298,9,69544
1149776,276706,0679447156,0,52540
1149777,276709,0515107662,10,15978


In [ ]:
df_rate.to_csv('newrating.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

### Junk Code

In [ ]:
# new_df = df_rate.groupby('ISBN').agg('BookRating').mean()
# new_df.columns = ['ISBN', 'avg_BookRating']
# new_df

In [ ]:
# df_rate['BookRating'] = df_rate['BookRating'].astype(str)
# df_rate = df_rate.groupby('UserID').agg(
#   ISBN=("ISBN", ",".join),
#   BookRating=("BookRating", ",".join)
# )
# df_rate

# Photo Detection

### Image to Text

In [ ]:
image_path = 'ImageTest/CM-1.jpg'
text = pytesseract.image_to_string(Image.open(image_path))
print(text)

classical

Mythology

Seventh Edition

MARK P.0. MORFORD @ ROBERT J. LENARDON

 



In [ ]:
# def ocr_core(fileimage):
#   text = pytesseract.image_to_string(Image.open(fileimage))

#   return text

# print(ocr_core('/content/ImageTest/CM-1.jpg'))

In [ ]:
text = re.sub(r'\n+', " ", text)
text = text.lower()
text = text.translate(str.maketrans("","",string.punctuation))

text = text.lstrip()
text = text.rstrip()
print(text)

classical mythology seventh edition mark p0 morford  robert j lenardon


In [ ]:
text = text.split(" ")
print(text)

['classical', 'mythology', 'seventh', 'edition', 'mark', 'p0', 'morford', '', 'robert', 'j', 'lenardon']


### Search Data from Text

In [ ]:
bookData = pd.DataFrame(df_book, columns= ['booktitle', 'bookauthor']).values.tolist()
bookData[:5]

[['Classical Mythology', 'Mark P. O. Morford'],
 ['Clara Callan', 'Richard Bruce Wright'],
 ['Decision in Normandy', "Carlo D'Este"],
 ['Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It',
  'Gina Bari Kolata'],
 ['The Mummies of Urumchi', 'E. J. W. Barber']]

In [ ]:
for i in range(len(bookData)):
  bookData[i] = str(bookData[i][0]) + " " + str(bookData[i][1])
  bookData[i] = bookData[i].lower()
  bookData[i] = bookData[i].translate(str.maketrans("","",string.punctuation))

bookData[:10]

['classical mythology mark p o morford',
 'clara callan richard bruce wright',
 'decision in normandy carlo deste',
 'flu the story of the great influenza pandemic of 1918 and the search for the virus that caused it gina bari kolata',
 'the mummies of urumchi e j w barber',
 'the kitchen gods wife amy tan',
 'what if the worlds foremost military historians imagine what might have been robert cowley',
 'pleading guilty scott turow',
 'under the black flag the romance and the reality of life among the pirates david cordingly',
 'where youll find me and other stories ann beattie']

In [ ]:
for i in range(len(bookData)):
  bookData[i] = bookData[i].split()

bookData[:3]

[['classical', 'mythology', 'mark', 'p', 'o', 'morford'],
 ['clara', 'callan', 'richard', 'bruce', 'wright'],
 ['decision', 'in', 'normandy', 'carlo', 'deste']]

In [ ]:
similarityList = []

index = 0

for book in bookData:
  similarity = 0
  for word in text:
    if word in book:
      similarity += 1
  similarityList.append([index, similarity])
  index += 1

similarityList[:10]

[[0, 4],
 [1, 0],
 [2, 0],
 [3, 0],
 [4, 1],
 [5, 0],
 [6, 1],
 [7, 0],
 [8, 0],
 [9, 0]]

In [ ]:
similarityList.sort(key=lambda x: x[1], reverse=True)
similarityList[:20]

#  [0, 95231, 1119777, 193923]

[[0, 4],
 [95231, 4],
 [111977, 4],
 [193923, 4],
 [42118, 3],
 [124762, 3],
 [127454, 3],
 [158903, 3],
 [196622, 3],
 [241341, 3],
 [264304, 3],
 [1087, 2],
 [1763, 2],
 [3179, 2],
 [3472, 2],
 [5008, 2],
 [7116, 2],
 [10299, 2],
 [10317, 2],
 [10663, 2]]

In [ ]:
output = [i[0] for i in similarityList[:4]]
output

[0, 95231, 111977, 193923]

# Recommendation Model

### Create Model

In [ ]:
print('books length : {}'.format(len(df_rate.bookid)))
print('users length : {}'.format(len(df_rate.userid)))
print('unique books length : {}'.format(len(df_rate.bookid.unique())))
print('unique users length : {}'.format(len(df_rate.userid.unique())))

books length : 1031136
users length : 1031136
unique books length : 270151
unique users length : 92106


In [ ]:
book_data = np.array(list(set(df_book.bookid)))
user_data = np.array([957 for i in range(len(book_data))])
length = len(book_data)

print('book data :')
print(book_data[:25])

print('user data :')
print(user_data[:5])

print('length : {}'.format(length))

book data :
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24]
user data :
[957 957 957 957 957]
length : 271360


In [ ]:
book_embedding_size = 8
user_embedding_size = 8

user_input = tf.keras.Input(shape=(1,), name='User-Input')
book_input = tf.keras.Input(shape=(1,), name='Book-Input')

user_embedded = tf.keras.layers.Embedding(df_rate.userid.max()+1, user_embedding_size, input_length=1, name='User-Embedding')(user_input)
book_embedded = tf.keras.layers.Embedding(df_rate.bookid.max()+1, book_embedding_size, input_length=1, name='Book-Embedding')(book_input)

concatenated = tf.keras.layers.Concatenate()([user_embedded, book_embedded])

out = tf.keras.layers.Flatten()(concatenated)
out = tf.keras.layers.Dense(128, activation='relu')(out)
out = tf.keras.layers.Dense(32, activation='relu')(out)
out = tf.keras.layers.Dense(1, activation='linear', name='prediction')(out)

model = tf.keras.Model(
    inputs = [user_input, book_input],
    outputs = out,
)
model.summary(line_length=88)

Model: "model_1"
________________________________________________________________________________________
Layer (type)                 Output Shape       Param #   Connected to                  
User-Input (InputLayer)      [(None, 1)]        0                                       
________________________________________________________________________________________
Book-Input (InputLayer)      [(None, 1)]        0                                       
________________________________________________________________________________________
User-Embedding (Embedding)   (None, 1, 8)       2230840   User-Input[0][0]              
________________________________________________________________________________________
Book-Embedding (Embedding)   (None, 1, 8)       2170880   Book-Input[0][0]              
________________________________________________________________________________________
concatenate_1 (Concatenate)  (None, 1, 16)      0         User-Embedding[0][0]          
    

In [ ]:
model.compile(
    tf.optimizers.Adam(0.005),
    loss='MSE',
    metrics=['MAE']
)

### Test Model

In [ ]:
train, test = train_test_split(df_rate, test_size=0.2, random_state=42)

In [ ]:
if os.path.exists('regression_model.h5'):
  model2 = tf.keras.models.load_model('regression_model.h5')
else:
  # history = model.fit([train.userid, train.bookid], train.bookrating, epochs=10, verbose=1)
  history = model.fit([df_rate.userid, df_rate.bookid], df_rate.bookrating, epochs=5, verbose=1)
  model.save('regression_model.h5')
  plt.plot(history.history['loss'])
  plt.xlabel("Epochs")
  plt.ylabel("Training Error")

In [ ]:
  model.evaluate([test.userid, test.bookid], test.bookrating)

6445/6445 [==============================] - 7s 1ms/step - loss: 22.8287 - MAE: 2.8375


[22.828710556030273, 2.837491035461426]

### Predict data

In [ ]:
predictions = model.predict([test.userid.head(10), test.bookid.head(10)])

[print(predictions[i], test.bookrating.iloc[i]) for i in range(0,10)]

[0.01960097] 5
[0.01919293] 7
[0.00743141] 0
[0.01103822] 4
[0.00897555] 10
[0.00282504] 0
[0.0137674] 0
[-0.00456957] 0
[0.00611231] 0
[0.01879044] 0


[None, None, None, None, None, None, None, None, None, None]

In [ ]:
predictions = model.predict([user_data, book_data])
predictions.shape

(271360, 1)

In [ ]:
predictions[:25]

array([[ 0.00943559],
       [-0.00054312],
       [ 0.00875589],
       [ 0.01199192],
       [ 0.00716341],
       [ 0.00295257],
       [ 0.00945492],
       [ 0.00753285],
       [ 0.00702061],
       [ 0.00444154],
       [ 0.01113363],
       [ 0.00145853],
       [ 0.00829699],
       [ 0.00910367],
       [ 0.00968371],
       [ 0.01728912],
       [ 0.00563196],
       [ 0.01320671],
       [-0.00017637],
       [ 0.01146057],
       [ 0.00828519],
       [ 0.00695769],
       [ 0.0061695 ],
       [ 0.00353427],
       [ 0.00542298]], dtype=float32)

In [ ]:
predictions = np.array([a[0] for a in predictions])
predictions[:25]

array([ 0.00943559, -0.00054312,  0.00875589,  0.01199192,  0.00716341,
        0.00295257,  0.00945492,  0.00753285,  0.00702061,  0.00444154,
        0.01113363,  0.00145853,  0.00829699,  0.00910367,  0.00968371,
        0.01728912,  0.00563196,  0.01320671, -0.00017637,  0.01146057,
        0.00828519,  0.00695769,  0.0061695 ,  0.00353427,  0.00542298],
      dtype=float32)

In [ ]:
recommended_book_id = (-predictions).argsort()[:25]    # Sorting + ambil 5 data terbesar
recommended_book_id

array([  5374,  81090, 132489,  64908, 150836, 192902,  70954, 168626,
       257707,  44791, 221239, 215133, 199771,  76495, 156825,  40696,
       205469, 250037, 170643, 151976,  95479,  67103, 212223, 225557,
       241632])

In [ ]:
predictions[recommended_book_id]

array([0.02398121, 0.02379338, 0.02378687, 0.02376336, 0.02375423,
       0.02372372, 0.02368725, 0.02362499, 0.023403  , 0.02328616,
       0.02324302, 0.02322474, 0.02307829, 0.0230304 , 0.02302613,
       0.0230165 , 0.02299728, 0.02297174, 0.02294293, 0.02290964,
       0.02281129, 0.02279934, 0.02273728, 0.02273217, 0.02272757],
      dtype=float32)

In [ ]:
df_book[df_book['bookid'].isin(recommended_book_id)]

,bookid,isbn,booktitle,bookauthor,yearofpublication,publisher,imageurls,imageurlm,imageurll,rating
5374,5374,0142000485,My Little Blue Dress,Bruno Maddox,2002,Penguin Books,http://images.amazon.com/images/P/0142000485.0...,http://images.amazon.com/images/P/0142000485.0...,http://images.amazon.com/images/P/0142000485.0...,2.812500
40696,40696,0465053548,The Origin of the Universe (Science Masters),John D. Barrow,1994,Perseus Books Group,http://images.amazon.com/images/P/0465053548.0...,http://images.amazon.com/images/P/0465053548.0...,http://images.amazon.com/images/P/0465053548.0...,0.000000
44791,44791,0156000024,Consider This Senora,Harriet Doerr,1994,Harvest Books,http://images.amazon.com/images/P/0156000024.0...,http://images.amazon.com/images/P/0156000024.0...,http://images.amazon.com/images/P/0156000024.0...,2.846154
64908,64908,037311933X,Secret Mother (Nanny Wanted!) (Harlequin Prese...,Justin Wilkinson,1997,Harlequin,http://images.amazon.com/images/P/037311933X.0...,http://images.amazon.com/images/P/037311933X.0...,http://images.amazon.com/images/P/037311933X.0...,0.000000
67103,67103,0312284330,Crashing the Party: How to Tell the Truth and ...,Ralph Nader,2001,St. Martin's Press,http://images.amazon.com/images/P/0312284330.0...,http://images.amazon.com/images/P/0312284330.0...,http://images.amazon.com/images/P/0312284330.0...,4.111111
70954,70954,0893755214,Surprise Party (Giant First-Start Reader),Sharon Gordon,1982,Troll Communications,http://images.amazon.com/images/P/0893755214.0...,http://images.amazon.com/images/P/0893755214.0...,http://images.amazon.com/images/P/0893755214.0...,0.000000
76495,76495,0870441973,Wonders of the desert world (Books for young e...,Judith E Rinard,1976,National Geographic Society,http://images.amazon.com/images/P/0870441973.0...,http://images.amazon.com/images/P/0870441973.0...,http://images.amazon.com/images/P/0870441973.0...,6.666667
81090,81090,0743227328,The Sex-Starved Marriage: A Couple's Guide to ...,Michele Weiner-Davis,2003,Simon &amp; Schuster,http://images.amazon.com/images/P/0743227328.0...,http://images.amazon.com/images/P/0743227328.0...,http://images.amazon.com/images/P/0743227328.0...,2.000000
95479,95479,0711991987,U2 Live: A Concert Documentary,Pimm Jal de la Parra,2003,Omnibus Press,http://images.amazon.com/images/P/0711991987.0...,http://images.amazon.com/images/P/0711991987.0...,http://images.amazon.com/images/P/0711991987.0...,10.000000
132489,132489,0765307871,Star Bright! : A Christmas Story,Andrew M. Greeley,2002,Forge Books,http://images.amazon.com/images/P/0765307871.0...,http://images.amazon.com/images/P/0765307871.0...,http://images.amazon.com/images/P/0765307871.0...,0.000000


### Input Output

In [ ]:
new_df = pd.DataFrame()
new_df['user_data'] = user_data
new_df['book_data'] = book_data
new_df['predictions'] = predictions
new_df

In [ ]:
new_df_2 = pd.DataFrame()
new_df_2['predictsort'] = list(predictions[recommended_book_id])
new_df_2['recom_book'] = list(recommended_book_id)
new_df_2[:10]

,predictsort,recom_book
0,[0.023981214],5374
1,[0.023793383],81090
2,[0.023786869],132489
3,[0.023763362],64908
4,[0.02375423],150836
5,[0.023723718],192902
6,[0.023687255],70954
7,[0.023624992],168626
8,[0.023403002],257707
9,[0.023286156],44791


In [ ]:
new_df[:10]

,user_data,book_data,predictions
0,957,0,0.009436
1,957,1,-0.000543
2,957,2,0.008756
3,957,3,0.011992
4,957,4,0.007163
5,957,5,0.002953
6,957,6,0.009455
7,957,7,0.007533
8,957,8,0.007021
9,957,9,0.004442


### Predict data loop

In [ ]:
array = []

for i in range(100):
  user_dataa = np.array([590])
  book_dataa = np.array([i])
  predictions = model.predict([user_dataa, book_dataa])
  array.append(predictions[0])

In [ ]:
array[:25]

[array([-0.00189273], dtype=float32),
 array([0.00220828], dtype=float32),
 array([-0.00098764], dtype=float32),
 array([0.00446855], dtype=float32),
 array([0.00064723], dtype=float32),
 array([0.00366151], dtype=float32),
 array([0.00585006], dtype=float32),
 array([0.0059635], dtype=float32),
 array([0.00886617], dtype=float32),
 array([0.00095742], dtype=float32),
 array([0.00051866], dtype=float32),
 array([0.00150454], dtype=float32),
 array([0.00432482], dtype=float32),
 array([-0.00071384], dtype=float32),
 array([0.00650441], dtype=float32),
 array([0.0081132], dtype=float32),
 array([-0.00017245], dtype=float32),
 array([-0.00013112], dtype=float32),
 array([0.00546825], dtype=float32),
 array([0.00280935], dtype=float32),
 array([-5.064658e-05], dtype=float32),
 array([0.01164684], dtype=float32),
 array([0.00349123], dtype=float32),
 array([0.00616914], dtype=float32),
 array([0.00586967], dtype=float32)]

### Deploy Model

In [ ]:
converter = lite.TFLiteConverter.from_keras_model(model)

In [ ]:
tfmodel = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmp2d62pwza/assets


INFO:tensorflow:Assets written to: /tmp/tmp2d62pwza/assets


In [ ]:
open('model.tflite', 'wb').write(tfmodel)

17634324

### Junk Code

In [ ]:
# Xtrain, Xtest = train_test_split(df_rate, test_size=0.2, random_state=1)
# print(f"Shape of train data: {Xtrain.shape}")
# print(f"Shape of test data: {Xtest.shape}")

In [ ]:
# #Get total unique user and book columns
# total_user = df_rate.userid.nunique()
# total_book = df_rate.bookid.nunique()

In [ ]:
# # Book input network
# book_input   = tf.keras.layers.Input(shape=[1])
# book_embed   = tf.keras.layers.Embedding(total_book+1, 15)(book_input)
# book_output  = tf.keras.layers.Flatten()(book_embed)

# # User input network
# user_input   = tf.keras.layers.Input(shape=[1])
# user_embed   = tf.keras.layers.Embedding(total_user+1, 15)(user_input)
# user_output  = tf.keras.layers.Flatten()(user_embed)

# # Concatenate, dense, and output layer
# conc_layer   = tf.keras.layers.Concatenate()([book_output, user_output])
# dense_layer  = tf.keras.layers.Dense(128, activation='relu')(conc_layer)
# output_layer = tf.keras.layers.Dense(1, activation='relu')(dense_layer)


# model = tf.keras.Model([book_input, user_input], output_layer)

In [ ]:
# opt = tf.keras.optimizers.Adam(learning_rate=0.001)
# model.compile(optimizer=opt, loss='mean_squared_error')
# model.summary()

In [ ]:
# Xtrain_array = [Xtrain.bookid, Xtrain.userid]
# Xtest_array = [Xtest.bookid, Xtest.userid]

In [ ]:
# hist = model.fit(Xtrain_array,
#                  Xtrain.bookrating, 
#                  batch_size=64, 
#                  epochs=5, 
#                  verbose=1,
#                  validation_data=(Xtest_array, Xtest.bookrating))

In [ ]:
# import tensorflow as tf
# import pandas as pd
# import numpy as np
# import re
# import string
# import csv

# import pytesseract
# import shutil
# import os
# import random
# try:
#   from PIL import Image
# except ImportError:
#   import Image

# print(tf.__version__)

In [ ]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# import warnings
# warnings.filterwarnings('ignore')
# from sklearn.model_selection import train_test_split